In [ ]:
# When using a colab notebook:
#!pip install aleph-alpha-client langchain python-dotenv

In [1]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

from langchain.llms import AlephAlpha

### Step 0: Define some helper functions

In [2]:
load_dotenv()

# for using the Aleph Alpha API
client = Client(token=os.getenv("AA_TOKEN"))

# for using LangChain
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), model="luminous-extended")

#### Step 1: Use the completion function of the API


In [4]:
# using the full-fledged AA API for a simple completion

# Define the prompt
prompt = Prompt.from_text("""### Instructions: Complete the following sentence with a few words.

### Input: An apple a day

### Response:""")

# Create a completion request with parameters
request = None # TODO: create a completion request with the prompt and parameters https://docs.aleph-alpha.com/docs/tasks/complete/ 

# Send the request to the API
response = client.complete(request=request, model="luminous-base-control")

# Get the completion
completion = response.completions[0].completion
print(completion)

PermissionError: [Errno 401] {"error":"The token used for authentication has been revoked. Replace it with a valid one. You\n            can list valid tokens in your users profile page.","code":"UNAUTHENTICATED"}

In [ ]:
# using LangChain for a simple completion

# define the prompt
prompt = "An apple a day"

# define the parameters
params = {
    "temperature": 0.5,
    "model": "luminous-extended",
    "maximum_tokens": 100
}

# define the model
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)

# get the completion
response = aleph_alpha(prompt=prompt, stop=["\n"])
print(response)

#### Step 2: Write a few shot prompt that generates keywords

##### Let's import some data to work on

In [ ]:
# Read the data in the data.md file
with open("data.md", "r", encoding="utf-8") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#####")

print(f"data: {data[:100]}")
print(f"texts: {texts[10][:100]}")

###### Let's create Keywords for the first 10 rows of the dataset

In [ ]:
# Get the first 10 texts
texts_for_keywords = texts

# TODO write a function that takes a text and returns a list of keywords
def get_keywords(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. TODO define the prompt
    prompt = Prompt.from_text(f"""A good Keyword prompt""")   
    
    # 2. define the completion request 
    request = CompletionRequest(prompt=prompt, 
                                    maximum_tokens=32, 
                                    temperature=0, 
                                    stop_sequences=["\n"])
    
    # 3. send the request to the API
    response = client.complete(request=request, model="luminous-base")
    
    # 4. get the completion
    completion = response.completions[0].completion
    return completion

def get_keywords_langchain(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. TODO define the prompt
    prompt = f"""Prompt"""
    
    # 2. define the parameters
    params = {
        "temperature": 0.5,
        "model": "luminous-base-control",
        "maximum_tokens": 100
    }

    # 3. define the model
    aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)
    
    # 4. get the completion
    response = aleph_alpha(prompt=prompt, stop=["\n"])
    return response
    

for text in texts:
    keywords = get_keywords(text)
    print(keywords)

### Let's use Langchain to run a chain

In [ ]:
# use a chain from langchain to generate a text

from langchain import PromptTemplate, LLMChain

# TODO Write a prompt that generates an answer to a question based on a text with the variables {context} and {question}
template = """{context}, {question}"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

llm = AlephAlpha(model="luminous-base-control", maximum_tokens=32, stop_sequences=["###:"], aleph_alpha_api_key=os.getenv("AA_TOKEN"))

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Give me a list of countries that include social elements."

answer = llm_chain.run(question = question, context = texts[4])

print(answer)

In [ ]:
# TODO write two prompts one, that generates a question about a text and one that then answers the questions

def generate_questions_and_answers(text):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
        
    # 1. TODO define the prompt
    questions_prompt = Prompt.from_text(f"""prompt""")
    
    # 2. TODO define the completion request for the questions
    request = CompletionRequest(prompt=questions_prompt, temperature=0.0, stop_sequences=["\n"])
    response = client.complete(request=request, model="luminous-base-control")
    question = response.completions[0].completion
    
    # 3. TODO get the completion for answers
    answers_prompt = Prompt.from_text(f"""prompt""")   
    request = CompletionRequest(prompt=answers_prompt, temperature=0.0, maximum_tokens=128)
    response = client.complete(request=request, model="luminous-base-control")
        
    # 4. get the completion
    completion = response.completions[0].completion
    
    return question, completion

text = texts[9]

question, completion = generate_questions_and_answers(text)
print(f"Question: {question}")
print(f"Answer: \n{completion}")

In [ ]:
# Let's write a function where the user chats with an AI, store the conversation in a list as a memory
history = []
def chat_with_ai(message):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
    history.append(f"User: {message}")
    breaker = "\n"
    # 1. TODO try varyinf the prompt
    prompt = Prompt.from_text(f"""### Instructions: ...

### History: 
{breaker.join(history)}

### AI:""")
    
    # 2. TODO define the completion request for the answer
    request = CompletionRequest(prompt=prompt, temperature=0.5,)
    response = client.complete(request=request, model="luminous-extended-control")
    answer = response.completions[0].completion
    
    
    
    history.append(f"AI: {answer}")
    
    return "\n".join(history)


In [ ]:
print(chat_with_ai("Maybe you can help me out. What are the most importnat things to know about boats?"))